In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Database connection parameters
server = '35.198.20.9'  # Your GCP SQL Server public IP
port = 1433             # Default SQL Server port
database = 'RiotDB'     # Your database name
username = 'sqlserver'  # Your SQL Server username
password = os.getenv('DB_PASSWORD')  # Retrieve password from environment variable

# Check if the password was loaded
if not password:
    raise ValueError("Database password not found. Please set the DB_PASSWORD environment variable.")

# Construct the database connection string
connection_url = (
    f"mssql+pyodbc://{username}:{password}@{server}:{port}/{database}"
    "?driver=ODBC+Driver+18+for+SQL+Server"
    "&Encrypt=yes"
    "&TrustServerCertificate=yes"
)

# Create the SQLAlchemy engine
engine = create_engine(connection_url)

# Function to calculate summoner win rate
def get_summoner_winrate(summoner_name):
    query = f"""
    SELECT 
        COUNT(CASE WHEN win = 1 THEN 1 END) AS Wins,
        COUNT(*) AS Total_Games
    FROM Detalhes_Liga_Invocador
    WHERE riotIdGameName = '{summoner_name}'
    """

    # Fetch data from the database
    df_winrate = pd.read_sql_query(query, engine)

    # Calculate win rate
    if df_winrate['Total_Games'].iloc[0] > 0:
        win_rate = (df_winrate['Wins'].iloc[0] / df_winrate['Total_Games'].iloc[0]) * 100
    else:
        win_rate = 0.0

    return round(win_rate, 2)

# Main function to run the process
if __name__ == "__main__":
    # Get the desired summoner name as input
    summoner_name = input("Enter the summoner's name: ")

    # Retrieve and display the win rate
    win_rate = get_summoner_winrate(summoner_name)
    print(f"\nWin Rate for {summoner_name}: {win_rate}%")



Win Rate for Mister Cricket: 50.0%
